In [8]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt

## Reading in the data and preparing DataFrames

#### MID

In [9]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
#print(mid.describe())
# #bring the time data into a new df:
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column

#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)
print(time)
time_sliced = time.iloc[:,11:].fillna(0)
print(time_sliced)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[2,0],format="%Y/%m/%d %H:%M:%S.%f")
    dic= {}
    itr = 0
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    key = "Timestamp, mass {}".format(masses[itr])
    for label,content in df2.items():
        key = "Timestamp, mass {}".format(masses[itr])
        #.strftime('%d-%m-%Y %H:%M:%S') not used
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=row*1000)))
        itr += 1
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df

               mass 2.0_time            mass 4.0_time  \
2    2022/12/13 17:34:18.708  2022/12/13 17:34:18.958   
3    2022/12/13 17:34:25.033  2022/12/13 17:34:25.283   
4    2022/12/13 17:34:31.358  2022/12/13 17:34:31.608   
5    2022/12/13 17:34:37.683  2022/12/13 17:34:37.933   
6    2022/12/13 17:34:44.008  2022/12/13 17:34:44.258   
..                       ...                      ...   
341  2022/12/13 18:10:02.883  2022/12/13 18:10:03.133   
342  2022/12/13 18:10:09.208  2022/12/13 18:10:09.458   
343  2022/12/13 18:10:15.533  2022/12/13 18:10:15.783   
344  2022/12/13 18:10:21.858  2022/12/13 18:10:22.108   
345  2022/12/13 18:10:28.183  2022/12/13 18:10:28.433   

               mass 5.5_time           mass 12.0_time  \
2    2022/12/13 17:34:19.208  2022/12/13 17:34:19.458   
3    2022/12/13 17:34:25.533  2022/12/13 17:34:25.783   
4    2022/12/13 17:34:31.858  2022/12/13 17:34:32.108   
5    2022/12/13 17:34:38.183  2022/12/13 17:34:38.433   
6    2022/12/13 17:34:44.508  

,"Timestamp, mass 2","Timestamp, mass 4","Timestamp, mass 5.5","Timestamp, mass 12","Timestamp, mass 15","Timestamp, mass 16","Timestamp, mass 18","Timestamp, mass 28","Timestamp, mass 32","Timestamp, mass 40","Timestamp, mass 44"
0,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358
1,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.684,2022-12-13 17:34:37.683,2022-12-13 17:34:31.358
2,2022-12-13 17:34:44.008,2022-12-13 17:34:44.008,2022-12-13 17:34:44.008,2022-12-13 17:34:44.008,2022-12-13 17:34:44.008,2022-12-13 17:34:44.009,2022-12-13 17:34:44.008,2022-12-13 17:34:44.008,2022-12-13 17:34:44.009,2022-12-13 17:34:44.008,2022-12-13 17:34:31.358
3,2022-12-13 17:34:50.333,2022-12-13 17:34:50.333,2022-12-13 17:34:50.333,2022-12-13 17:34:50.333,2022-12-13 17:34:50.333,2022-12-13 17:34:50.334,2022-12-13 17:34:50.333,2022-12-13 17:34:50.333,2022-12-13 17:34:50.334,2022-12-13 17:34:50.333,2022-12-13 17:34:31.358
4,2022-12-13 17:34:56.658,2022-12-13 17:34:56.658,2022-12-13 17:34:56.658,2022-12-13 17:34:56.658,2022-12-13 17:34:56.658,2022-12-13 17:34:56.659,2022-12-13 17:34:56.658,2022-12-13 17:34:56.658,2022-12-13 17:34:56.659,2022-12-13 17:34:56.658,2022-12-13 17:34:31.358
...,...,...,...,...,...,...,...,...,...,...,...
339,2022-12-13 18:10:15.533,2022-12-13 18:10:15.533,2022-12-13 18:10:15.533,2022-12-13 18:10:15.533,2022-12-13 18:10:15.533,2022-12-13 18:10:15.534,2022-12-13 18:10:15.533,2022-12-13 18:10:15.533,2022-12-13 18:10:15.534,2022-12-13 18:10:15.533,2022-12-13 17:34:31.358
340,2022-12-13 18:10:21.858,2022-12-13 18:10:21.858,2022-12-13 18:10:21.858,2022-12-13 18:10:21.858,2022-12-13 18:10:21.858,2022-12-13 18:10:21.859,2022-12-13 18:10:21.858,2022-12-13 18:10:21.858,2022-12-13 18:10:21.859,2022-12-13 18:10:21.858,2022-12-13 17:34:31.358
341,2022-12-13 18:10:28.183,2022-12-13 18:10:28.183,2022-12-13 18:10:28.183,2022-12-13 18:10:28.183,2022-12-13 18:10:28.183,2022-12-13 18:10:28.184,2022-12-13 18:10:28.183,2022-12-13 18:10:28.183,2022-12-13 18:10:28.184,2022-12-13 18:10:28.183,2022-12-13 17:34:31.358
342,2022-12-13 18:10:34.508,2022-12-13 18:10:34.508,2022-12-13 18:10:34.508,2022-12-13 18:10:34.508,2022-12-13 18:10:34.508,2022-12-13 18:10:34.509,2022-12-13 18:10:34.508,2022-12-13 18:10:34.508,2022-12-13 18:10:34.509,2022-12-13 18:10:34.508,2022-12-13 17:34:31.358


#### Vaclogger

In [10]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
0,NaN,2022-12-13 17:31:14,0.0,-0.046295,2.540000e-09,1.640000e-09,4.996000e-09,4.919000e-07,0.35246,197.0,-5.163,4288.086,NaN,NaN
1,NaN,2022-12-13 17:31:21,7.0,-0.047451,2.900000e-09,1.680000e-09,4.996000e-09,4.919000e-07,0.35245,197.0,-5.275,4288.198,NaN,NaN
2,NaN,2022-12-13 17:31:28,14.0,-0.046614,2.900000e-09,1.640000e-09,4.996000e-09,4.919000e-07,0.35244,197.0,-5.050,4286.403,NaN,NaN
3,NaN,2022-12-13 17:31:35,21.0,-0.050953,2.400000e-09,1.860000e-09,4.996000e-09,4.733000e-07,0.35245,196.0,-5.275,4286.739,NaN,NaN
4,NaN,2022-12-13 17:31:42,28.0,-0.046468,1.140000e-09,6.030000e-10,4.996000e-09,4.688000e-07,0.35244,196.0,-5.163,4286.515,NaN,NaN


#### HiVolta log

In [11]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,hv_grid,em
0,2022-12-13 17:30:03,13/12/2022,0.000,17:30:03.498,0.06,0.1,0.18,0.18,0.0,0.0,...,0.0062,0.0129,0.0105,0.0167,-0.0142,-0.0258,-0.0100,NaN,1.000000e-10,6.200000e-09
1,2022-12-13 17:30:04,13/12/2022,1.178,17:30:04.676,0.06,0.1,0.18,0.18,0.0,0.0,...,0.0062,0.0129,0.0105,0.0164,-0.0142,-0.0258,-0.0100,NaN,1.000000e-10,6.200000e-09
2,2022-12-13 17:30:05,13/12/2022,2.366,17:30:05.864,0.06,0.1,0.18,0.18,0.0,0.0,...,0.0062,0.0129,0.0105,0.0101,-0.0142,-0.0258,-0.0100,NaN,1.000000e-10,6.200000e-09
3,2022-12-13 17:30:07,13/12/2022,3.548,17:30:07.046,0.06,0.1,0.18,0.18,0.0,0.0,...,0.0062,0.0129,0.0105,0.0171,-0.0142,-0.0258,-0.0100,NaN,1.000000e-10,6.200000e-09
4,2022-12-13 17:31:58,13/12/2022,114.845,17:31:58.343,0.06,0.1,0.14,0.16,0.0,0.0,...,0.0057,0.0165,0.0082,0.0124,-0.0143,-0.0262,-0.0099,NaN,2.000000e-10,5.700000e-09


## Checking injection volume pressure drop linearity

In [12]:
#defining a function for the straight line
def fit(x,a,b):
    return a*x + b
#curve fit for the data
y_val = vaclog["injection 1mbar"].values
x_val = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(fit,x_val,y_val)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.00*x+0.35


## Data processing

In [13]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan]


In [216]:
#print hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv["Comment"].dropna()
hv_comm.index

[nan 'no emission' 'emission on' 'minimal emission current'
 'grid repelling' 'grid transparent' 'ega turned' 'grid repelling again'
 'no electron activity']


Int64Index([  17,   18,   19,   20,   50,   51,   52,   53,  339,  340,  341,
             342,  652,  653,  654,  655,  660,  661,  662,  663,  776,  777,
             778,  779,  834,  835,  836,  837, 1359, 1360, 1361, 1362, 1857,
            1858, 1859, 1860],
           dtype='int64')

In [217]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


### Data preparation for calculating desorption yield from first ESD spike

In [16]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
print("Em on:",hv.loc[hv["Comment"]=="emission on"])
print("Grid repelling:", hv.loc[hv["Comment"]=="grid repelling"])
x=pd.Timestamp(2022, 12, 13, 17,38,53)
y=pd.Timestamp(2022, 12, 13, 17, 45, 21)

#then find the time value closest to it in the MID scan file 
print("\n","Em on in MID file:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
print("\n","Grid repelling in MID file:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 5.5_value"])

#apply electronic background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck

#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #A/mbar
h2_equiv = 0.46 #H2 equivalent pressures from N2 reading
p_h2 = mid["mass 2, corr"]/(h2_sens*h2_equiv) 

#account for the thermal transpiration factor
T2 = 4.2
T1 = 293
T_coef = np.sqrt(T2/T1)
em_h2 = p_h2*T_coef 

#baseline and emission pp for partial pressure delta
em_h2 = p_h2[44:57]
print(em_h2.head())
base_h2=mid["mass 2, corr"][0:44]/h2_sens
base_h2.head()

Em on:                datetime        Date  Elapsed time          Time  VMon1  \
339 2022-12-13 17:38:53  13/12/2022       529.855  17:38:53.353   0.08   
340 2022-12-13 17:38:54  13/12/2022       531.038  17:38:54.536   0.10   
341 2022-12-13 17:38:55  13/12/2022       532.227  17:38:55.725   0.16   
342 2022-12-13 17:38:56  13/12/2022       533.471  17:38:56.969   0.08   

      VMon2   VMon3   VMon4   VMon5  VMon6  ...   IMon2    IMon3    IMon4  \
339  100.12  999.66  999.66  641.82    0.0  ...  0.3700  26.3643  26.3879   
340  100.12  999.66  999.66  771.98    0.0  ...  0.3670  27.7330  27.7084   
341  100.12  999.66  999.64  902.14    0.0  ...  0.3773  28.6768  28.6777   
342  100.12  999.68  999.70  998.46    0.0  ...  0.4532  29.8831  29.8351   

       IMon5   IMon6   IMon7   IMon8      Comment       hv_grid            em  
339  26.3281 -0.0149 -0.0271 -0.0114  emission on  6.000000e-10  3.700000e-07  
340  27.6982 -0.0149 -0.0271 -0.0114  emission on  2.900000e-09  3.670000e-0

2    9.013870e-10
3    9.124621e-10
4    9.236170e-10
5    9.313400e-10
6    9.256593e-10
Name: mass 2, corr, dtype: float64

## Apparent desorption yield calculation for H2
### Formula for the cryo setup
$$
\eta_{app}=\frac{C_{comb}\cdot (p_{n,EM peak} - \overline{p}_{base}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$

T here is @RT and therefore thermal transpiration compensation should be applied afterwards to $\Delta p$.

$C_{comb}$ takes into account the conductance from the sample via the copper tube for H2 and the pumping speed for turbo pump

In [17]:
#calculation of delta_p-s
dp1 = (np.mean(em_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = scipy.constants.value("Boltzmann constant")
T = 293
q_e = scipy.constants.value("elementary charge")
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)
#Emission current
#ignore the inrush current values
hv["em"].where(hv["em"] <= 1.5e-6, np.nan, inplace=True)

I_em = np.nanmean(hv["em"].nlargest(50))
print(I_em)



1.1711001626994413e-07
0.01992516581153798
4.83732e-07


In [220]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("S1 The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

S1 The desorption yield for H2 [mol/el]: 0.004196183563079405 



## Electron dose


Find:

$$
Q = \int_{t_1}^{t_2} I \,dt
$$

Where $t_2$ and $t_1$ are the timestamps at which an emission current reading was taken


In [ ]:
#Integrating the emission current
def integrand(t, i):
    return i*t

i = I_n.dropna().reset_index() 

#for testing:
#t2_2 = (hv_aligned["Elapsed time"].diff(periods=1).fillna(0))

#adding an elapsed time column
timestamp3=[]
for j in hv["Time"]:
    timestamp3.append(j)
    
#making the column into a datetime object
timestamps3=pd.to_datetime(i["datetime"],format="%H:%M:%S")
runtime3=(timestamps3-timestamps3[0]).dt.total_seconds()    
i.insert(2,"Elapsed time",runtime3)

#define the n, n-1 difference in a variable "t2"
t2 = (i["Elapsed time"].diff(periods=1).fillna(0))

#replace all values from t2 that are further away than 50% of the median (.. > 9) with the median value
t2.where(t2 <= 12,6,inplace=True)
print(t2)

#integration
ar = []
for x in t2:
    itr = 0
    I = quad(integrand, 0, x, args=(i["I_em"][itr]))
    ar.append(I[0])
    itr += 1
    #if previous value was not NaN, replace the value with the median of elapsed time
    #if 
    

In [ ]:
#calculate the cumulative charge
Q_cumul = np.cumsum(ar)
Q_sum = sum(ar)
print("Total charge emitted from EGA during ~5000 seconds:", f"{Q_sum:.4E}", "C")

#cumulative nr of electrons emitted
N_e = Q_sum/1.6e-19
Ne_cumul = Q_cumul/1.6e-19
print("Total number of electron emitted during ~5000 seconds:", f"{N_e:.4E}", "e")
      
#To get the electron dose, divide n of electrons by sample surface area of 276 cm2, multiply by grid transmittance of 0.9
tot_dose = (N_e/276)*0.9
cumul_dose = (Ne_cumul/276)*0.9
print("Total electron dose on the sample ~5000 seconds:", f"{tot_dose:.4E}", "e-/cm^2")
      
#add these to the "i" dataframe:
i.insert(3,"Cumulative charge", Q_cumul)
i.insert(4,"Cumulative n. of electrons emitted from EGA", Ne_cumul)
i.insert(5,"Cumulative electron dose on the HL sample", cumul_dose)
i.set_index("datetime", drop=True, inplace=True)

#Ignore the first value, also, match the shape of eta
i = i.loc[i.index[1]:]
print(i)


Total charge emitted from EGA during ~5000 seconds: 6.6102E-02 C
Total number of electron emitted during ~5000 seconds: 4.1313E+17 e
Total electron dose on the sample ~5000 seconds: 1.3472E+15 e-/cm^2
                             I_em  Elapsed time  Cumulative charge  \
datetime                                                             
2023-02-24 14:30:22  3.298600e-06           5.0           0.000041   
2023-02-24 14:30:27  3.294800e-06          10.0           0.000083   
2023-02-24 14:30:31  3.293100e-06          14.0           0.000109   
2023-02-24 14:30:36  3.288200e-06          19.0           0.000151   
2023-02-24 14:30:41  3.276700e-06          24.0           0.000192   
...                           ...           ...                ...   
2023-02-24 17:18:04  1.064000e-07       10067.0           0.065936   
2023-02-24 17:18:09  1.064000e-07       10072.0           0.065977   
2023-02-24 17:18:14  1.064000e-07       10077.0           0.066019   
2023-02-24 17:18:19  1.064000

## Plotting

In [18]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [222]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(new_df["Timestamp, mass 4"],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(new_df["Timestamp, mass 12"],mid["mass 12.0_value"],marker=".", markersize=4,label='M12')
plt.plot(new_df["Timestamp, mass 15"],mid["mass 15.0_value"],marker=".", markersize=4,label='M16')
plt.plot(new_df["Timestamp, mass 16"],mid["mass 16.0_value"],marker=".", markersize=4,label='M15')
plt.plot(new_df["Timestamp, mass 18"],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(new_df["Timestamp, mass 28"],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(new_df["Timestamp, mass 32"],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(new_df["Timestamp, mass 40"],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
# plt.plot(new_df["Timestamp, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Timestamp')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S1 MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv["datetime"][i],0.95,str(hv["Comment"][i]),fontdict=font, transform=trans)


plt.savefig(r'./graphs/MID.png')
plt.show()


In [223]:
#plotting HiVolta log
plt.figure(figsize=(12,6))

plt.plot(hv["datetime"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["datetime"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Timestamp')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("S1 HiVolta log")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv["datetime"][i],0.8,str(hv["Comment"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/HiVolta.png')
plt.show()


In [224]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/Barions.png')
plt.show()

In [225]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S1 Temperature evolution")
plt.yscale('linear')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()

In [226]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots(figsize=(10,6))
y_fit = fit(x_val,a,b)
ax.plot((vaclog["Time"]),vaclog["injection 1mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.legend(loc="upper right")
ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
ax.get_yaxis().get_major_formatter().set_useOffset(False)
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

ax2 = ax.twiny()
ax2.plot((vaclog["Elapsed time"]),y_fit, "--",color="red", label="Y= -ax + b fit") 
ax2.legend(loc="lower left")
ax2.set_xlabel('Elapsed time (s)')
ax2.set_ylabel('Pressure (mbar)')
ax2.set_yscale('linear')
ax2.get_yaxis().get_major_formatter().set_useOffset(False)

plt.title("S1 Injection pressure evolution 1mbar conductance gauge")
plt.savefig(r'./graphs/Injection.png')
plt.show()

In [ ]:
#plotting esd vs electron dose

## Writing edited dataframes to file

In [227]:
#new_df together with MID ion current values
#take rows 1,3,5,7,9.. from new_df and rows 2,4,6,8.. from MID
mid_signal = pd.DataFrame(mid.iloc[:,1::2])
comb = pd.concat([new_df, mid_signal], axis=1)
comb = comb[[item for items in zip(new_df.columns, mid_signal.columns) for item in items]]
comb.to_csv(r'./data analysis/MID_output.csv', sep='\t', encoding='utf-8', header=True)


In [228]:
#desorption yield params
esd_frame = {"p_H2 during ESD":em_h2, "H2 base pressure":base_h2, "conductance factor":x_h2,"Temp":T}
esd = pd.DataFrame(esd_frame)
print(esd)
esd.to_csv(r'./data analysis/esd_output.csv', sep='\t', encoding='utf-8', header=True)


    p_H2 during ESD  H2 base pressure  conductance factor  Temp
2               NaN      9.013870e-10            0.019925   293
3               NaN      9.124621e-10            0.019925   293
4               NaN      9.236170e-10            0.019925   293
5               NaN      9.313400e-10            0.019925   293
6               NaN      9.256593e-10            0.019925   293
7               NaN      9.351757e-10            0.019925   293
8               NaN      9.339644e-10            0.019925   293
9               NaN      9.348987e-10            0.019925   293
10              NaN      9.393119e-10            0.019925   293
11              NaN      9.403494e-10            0.019925   293
12              NaN      9.354011e-10            0.019925   293
13              NaN      9.373729e-10            0.019925   293
14              NaN      9.425184e-10            0.019925   293
15              NaN      9.487015e-10            0.019925   293
16              NaN      9.532696e-10   

In [229]:
#hivolta log
hv.to_csv(r'./data analysis/HV_output.csv', sep='\t', encoding='utf-8', header=True)

In [230]:
#vaclog
vaclog.to_csv(r'./data analysis/vaclogger_output.csv', sep='\t', encoding='utf-8', header=True)

In [231]:
#CERNoX
temps_frame = {"Timestamp":vaclog["Time"],"Temperatures":polyfit(A,vaclog["T-CERNOX"])}
temps = pd.DataFrame(temps_frame)
temps.to_csv(r'./data analysis/CERNoX temps.csv', sep='\t', encoding='utf-8', header=True)